In [1]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
secret_label = "HF_TOKEN"
secret_value = UserSecretsClient().get_secret(secret_label)
login(secret_value)

In [2]:
import gc
import torch

def clear_cuda():
    gc.collect()                    # Python garbage collection
    if torch.cuda.is_available():
        torch.cuda.empty_cache()    # Releases unreferenced memory back to the pool
        torch.cuda.ipc_collect()    # Cleans up inter-process memory
#clear_cuda()

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

def load_model(model_name):
    print(f"Loading model: {model_name}")
    clear_cuda()
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if "gemma" in model_name.lower():
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype="float16", #float16 for gemma idk why auto doesnt work
            device_map="auto",          
        )
    else:
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype="auto", #float16 for gemma idk why auto doesnt work
            device_map="auto",          
        )
    print("Model successfully loaded across devices:")
    for name, param in model.named_parameters():
       if param.device.type == 'cuda': 
           print(f" - {name} on {param.device}")
           break  # print only one to verify
    return tokenizer, model
#tokenizer, model = load_model(model_name)

In [4]:
import pandas as pd
label = pd.read_csv('/kaggle/input/hs-history-2024/labeled-history-202x-dedup.csv')
label.drop('Unnamed: 0', axis=1, inplace=True)
label = label[label['dang_cau_hoi'].notna()].reset_index(drop=True)
label = label.rename(columns={'options': 'choices'})
label

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,answer,question_type,grade,choices,question,nhan_biet,thong_hieu,van_dung,van_dung_cao,dang_cau_hoi
0,history-2022-302-35,A,multiple_choice,12.0,"['Có sự linh hoạt, sáng tạo trong phương án tá...",Chiến dịch Biên giới thu-đông (1950) và chiến ...,NaN,NaN,1.0,NaN,"So sánh, đối chiếu"
1,history-2022-304-37,A,multiple_choice,12.0,['Sử dụng đòn tiến công chiến lược của lực lượ...,Chiến dịch Biên giới thu-đông (1950) và chiến ...,NaN,NaN,1.0,NaN,"So sánh, đối chiếu"
2,history-2020-304-21,A,multiple_choice,12.0,"['sự bất lực của quân đội Sài Gòn', 'nhiệm vụ ...",Chiến thắng Phước Long (tháng 1-1975) của quân...,NaN,NaN,1.0,NaN,Đánh giá
3,history-2020-301-32,A,multiple_choice,12.0,"['khả năng thắng lớn của quân giải phóng.', 'q...",Chiến thắng Phước Long (tháng 1-1975) của quân...,NaN,NaN,1.0,NaN,Đánh giá
4,history-2020-302-23,A,multiple_choice,12.0,"['sức mạnh to lớn của quân giải phóng.', 'quân...",Chiến thắng Phước Long (tháng 1-1975) của quân...,NaN,NaN,1.0,NaN,Đánh giá
...,...,...,...,...,...,...,...,...,...,...,...
555,history-2021-304-9,A,multiple_choice,12.0,"['Thực dân Pháp bị thiệt hại nặng nề', 'Nhật đ...","Ở Đông Dương, thực dân Pháp đề ra kế hoạch quâ...",NaN,1.0,NaN,NaN,Bối cảnh/Nguyên nhân
556,history-2021-301-11,C,multiple_choice,12.0,"['Quân phiệt Nhật Bản tiến vào Đông Dương.', '...","Ở Đông Dương, thực dân Pháp đề ra kế hoạch quâ...",NaN,1.0,NaN,NaN,Bối cảnh/Nguyên nhân
557,history-2021-302-3,B,multiple_choice,12.0,"['Quân phiệt Nhật Bản tiến vào Đông Dương.', '...","Ở Đông Dương, thực dân Pháp đề ra kế hoạch quâ...",NaN,1.0,NaN,NaN,Bối cảnh/Nguyên nhân
558,history-2021-303-2,C,multiple_choice,12.0,"['Quân Trung Hoa Dân quốc tiến vào Việt Nam.',...","Ở Đông Dương, thực dân Pháp đề ra kế hoạch quâ...",NaN,1.0,NaN,NaN,Bối cảnh/Nguyên nhân


In [5]:
def prompter(question, choices):
    if isinstance(choices, str):
        choices_list = "\n".join([f"{chr(65+i)}. {c}" for i,c in enumerate(ast.literal_eval(choices))])
    else:
        choices_list = "\n".join([f"{chr(65+i)}. {c}" for i,c in enumerate(choices)])
    prompt = f"""
### Bạn sẽ được cung cấp một câu hỏi về lịch sử. Chọn câu trả lời đúng và đặt trong ngoặc vuông như: [A], [B], [C] hoặc [D]. Hãy suy nghĩ từng bước một.
**Không** lặp lại câu hỏi hoặc lựa chọn. Không cần giải thích lại đáp án.
Nếu bạn muốn giải thích, hãy trả lời câu hỏi bằng chữ cái [A], [B], [C] hoặc [D] trước, sau đó giải thích câu trả lời của bạn.

### VÍ DỤ MẪU
    
---

Câu hỏi: Chiến thắng Điện Biên Phủ (1954) và thắng lợi của trận “Điện Biên Phủ trên không” (1972) ở Việt Nam đều cho thấy
A. giá trị của trận quyết chiến chiến lược đánh dấu kháng chiến kết thúc thắng lợi
B. vai trò quyết định của mặt trận quân sự trong mối quan hệ với chính trị và ngoại giao
C. ý nghĩa chiến lược của trận phản công lớn nhất trong cuộc chiến tranh bảo vệ Tổ quốc
D. sức mạnh của quân chủ lực khi tấn công vào tổ chức phòng ngự mạnh của đối phương

Answer: [B] Cả hai chiến thắng này đều mang tính quyết định trong việc buộc đối phương phải chấp nhận giải pháp chính trị - ngoại giao bất lợi cho họ. Điều đó chứng tỏ mặt trận quân sự có vai trò quyết định, làm cơ sở cho thắng lợi chính trị và ngoại giao.

---

### Trả lời câu hỏi lịch sử sau đây:
{question}
### Đây là các lựa chọn:
{choices_list}
### Answer: """
    return prompt

In [6]:
def prompter_en(question, choices):
    if isinstance(choices, str):
        choices_list = "\n".join([f"{chr(65+i)}. {c}" for i,c in enumerate(ast.literal_eval(choices))])
    else:
        choices_list = "\n".join([f"{chr(65+i)}. {c}" for i,c in enumerate(choices)])
    prompt = f"""
### You are given a history question. Choose the correct answer and put it inside square brackets like: [A], [B], [C], or [D]. Let's think step by step.
Do **NOT** repeat the question or choices. You do NOT need explain your answer. 
If you are going to explain yourself, answer the question with a letter [A], [B], [C] or [D] first, then explain your answer.
### Answer this history question:
{question}
### Here are your choices:
{choices_list} 
### Answer: """
    return prompt

In [5]:
import ast
def is_chat_model(model_name):
    return any(x in model_name.lower() for x in ["chat", "qwen", "gemma", "llama", "vistral"])

def is_vietnamese_model(model_name):
    return any(x in model_name.lower() for x in ["vinallama", "vistral", "viet", "vi-"])

def build_fewshot_cot_prompt(model_name, question, choices):
    if isinstance(choices, str):
        choices_list = "\n".join([f"{chr(65+i)}. {c}" for i,c in enumerate(ast.literal_eval(choices))])
    else:
        choices_list = "\n".join([f"{chr(65+i)}. {c}" for i,c in enumerate(choices)])

    fewshot_cot_vi = f"""
    Bạn sẽ được cung cấp câu hỏi lịch sử cấp trung học phổ thông với các lựa chọn của câu hỏi đó.
    Suy nghĩ từng bước một và trả lời câu hỏi được cho bằng một chữ cái [A], [B], [C] hay [D]. 
    Hãy trả lời đáp án theo định dạng giống với ví dụ trước khi giải thích.
    
    ### MỤC TIÊU
    Phân tích câu hỏi, và suy nghĩ từng bước một. Đưa ra MỘT câu trả lời đúng trong bốn lựa chọn được cho và đặt chữ cái của câu trả lời trong ngoặc vuông như sau: `[A]`, `[B]`, `[C]` HOẶC `[D]`. 

    ## QUY TẮC BẮT BUỘC.
    1. **KHÔNG** được lặp lại câu hỏi hoặc các lựa chọn. Quy tắc này phải tuân thủ **NGHIÊM NGẶT**.
    2. Nếu bạn có định giải thích, hãy trả lời bằng đáp án bạn chọn trước khi giải thích. 
    3. Đưa ra MỘT đáp án cuối cùng. Đáp án này PHẢI đưa ra TRƯỚC khi bạn giải thích đáp án.
    4. Học cách trả lời và đưa ra lập luận theo các ví dụ ở dưới.
    
    ### VÍ DỤ MẪU
    
    ---

    Câu hỏi: Chiến thắng Điện Biên Phủ (1954) và thắng lợi của trận “Điện Biên Phủ trên không” (1972) ở Việt Nam đều cho thấy
    A. giá trị của trận quyết chiến chiến lược đánh dấu kháng chiến kết thúc thắng lợi
    B. vai trò quyết định của mặt trận quân sự trong mối quan hệ với chính trị và ngoại giao
    C. ý nghĩa chiến lược của trận phản công lớn nhất trong cuộc chiến tranh bảo vệ Tổ quốc
    D. sức mạnh của quân chủ lực khi tấn công vào tổ chức phòng ngự mạnh của đối phương

    Answer: [B] Cả hai chiến thắng này đều mang tính quyết định trong việc buộc đối phương phải chấp nhận giải pháp chính trị - ngoại giao bất lợi cho họ. Điều đó chứng tỏ mặt trận quân sự có vai trò quyết định, làm cơ sở cho thắng lợi chính trị và ngoại giao.
    
    ---
    
    Câu hỏi: Chiến thắng Điện Biên Phủ năm 1954 của quân dân Việt Nam đã làm phá sản hoàn toàn kế hoạch nào sau đây của thực dân Pháp?
    A. Rove
    B. Bôlae
    C. Đờ Lát đơ Tátxinhi
    D. Nava

    Answer: [D] Điện Biên Phủ làm phá sản kế hoạch Nava – kế hoạch quân sự lớn cuối cùng của Pháp nhằm giành thắng lợi quân sự quyết định trong 18 tháng. Các kế hoạch Rove, Bôlae, Đờ Lát đều đã bị đánh bại trước đó.
    
    ---
    
    Câu hỏi: Chiến thắng Đường 14-Phước Long (đầu năm 1975) có tác động nào sau đây đối với tiến trình cách mạng miền Nam Việt Nam?
    A. Củng cố quyết tâm chiến lược giải phóng hoàn toàn miền Nam
    B. Chuyển cách mạng miền Nam từ thế giữ gìn lực lượng sang thế tiến công
    C. Lần đầu khẳng định bạo lực là con đường giải phóng miền Nam
    D. Buộc Mĩ xuống thang chiến tranh và chấp nhận đàm phán tại Hội nghị Pari

    Answer: [A] Phước Long là tỉnh đầu tiên được giải phóng hoàn toàn và Mỹ không có hành động can thiệp trở lại. Điều này chứng tỏ Mỹ đã suy yếu, tạo niềm tin và củng cố quyết tâm chiến lược tiến tới giải phóng hoàn toàn miền Nam.
    
    ---

    Câu hỏi: Chiến tranh thế giới thứ hai (1939 – 1945) bùng nổ là do nguyên nhân nào sau đây?
    A. Tác động từ sự tan rã của trật tự thế giới hai cực Ianta
    B. Sự xuất hiện, hoạt động của Tổ chức Hiệp ước Vácsava
    C. Sự xuất hiện của Tổ chức Hiệp ước Bắc Đại Tây Dương
    D. Mâu thuẫn giữa các nước đế quốc về vấn đề thuộc địa

    Answer: [D] Nguyên nhân sâu xa là mâu thuẫn gay gắt giữa các nước đế quốc về vấn đề thị trường và thuộc địa. Các phương án A, B, C liên quan tới giai đoạn sau 1945, không đúng bối cảnh bùng nổ Thế chiến II.
    
    ---

    Câu hỏi: Cương lĩnh chính trị (đầu năm 1930) của Đảng Cộng sản Việt Nam... phản ánh đúng quan điểm nào sau đây của Nguyễn Ái Quốc?
    A. Lấy nhiệm vụ dân chủ làm nền tảng để giải quyết nhiệm vụ dân tộc
    B. Giải quyết hài hòa mối quan hệ giữa vấn đề dân tộc và vấn đề giai cấp
    C. Xác định các tầng lớp của giai cấp bóc lột đều là kẻ thù của cách mạng
    D. Tiến hành đồng thời cách mạng phản đế và cách mạng điền địa

    Answer: [B] Cương lĩnh đã xác định vừa giải phóng dân tộc khỏi ách đế quốc, vừa thực hiện các nhiệm vụ ruộng đất, phản ánh quan điểm giải quyết đồng thời và hài hòa giữa vấn đề dân tộc và vấn đề giai cấp.
    
    ---
    
    Trả lời câu hỏi lịch sử sau: {question}
    
    Các phương án trả lời: {choices_list}
    
    Answer:""".strip()
    
    fewshot_cot_en = f"""
    You are a history expert with deep knowledge. 
    You will be given a high school history multiple-choice question with four options.  
    If you provide reasoning, **always give the answer first, then the explanation**.

    ### OBJECTIVE
    Analyze the question step by step. Provide ONLY ONE correct answer among the four options, and put the letter inside square brackets like: `[A]`, `[B]`, `[C]`, or `[D]`. 

    ### STRICT RULES
    1. **DO NOT** repeat the question or the options. Follow this rule strictly.  
    2. If you explain, state the answer you choose first, then the reasoning.  
    3. Think carefully and give ONLY ONE final answer.  
    4. Follow the sample examples below and answer in the same format.  

    ### SAMPLE EXAMPLES
    
    ---

    Question: Chiến thắng Điện Biên Phủ (1954) và thắng lợi của trận “Điện Biên Phủ trên không” (1972) ở Việt Nam đều cho thấy  
    A. giá trị của trận quyết chiến chiến lược đánh dấu kháng chiến kết thúc thắng lợi  
    B. vai trò quyết định của mặt trận quân sự trong mối quan hệ với chính trị và ngoại giao  
    C. ý nghĩa chiến lược của trận phản công lớn nhất trong cuộc chiến tranh bảo vệ Tổ quốc  
    D. sức mạnh của quân chủ lực khi tấn công vào tổ chức phòng ngự mạnh của đối phương  
    
    Answer: [B] Both victories were decisive in forcing the opponent to accept unfavorable political and diplomatic solutions. This shows that the military front played the decisive role, laying the foundation for political and diplomatic victories.  

    ---

    Question: Chiến thắng Điện Biên Phủ năm 1954 của quân dân Việt Nam đã làm phá sản hoàn toàn kế hoạch nào sau đây của thực dân Pháp?  
    A. Rove 
    B. Bôlae 
    C. Đờ Lát đơ Tátxinhi 
    D. Nava 
    
    Answer: [D] Điện Biên Phủ destroyed the Navarre Plan—the last major French military strategy aimed at achieving decisive victory within 18 months. The Rove, Bôlae, and De Lattre plans had already been defeated earlier.  

    ---

    Question: Chiến thắng Đường 14-Phước Long (đầu năm 1975) có tác động nào sau đây đối với tiến trình cách mạng miền Nam Việt Nam?  
    A. Củng cố quyết tâm chiến lược giải phóng hoàn toàn miền Nam  
    B. Chuyển cách mạng miền Nam từ thế giữ gìn lực lượng sang thế tiến công  
    C. Lần đầu khẳng định bạo lực là con đường giải phóng miền Nam  
    D. Buộc Mĩ xuống thang chiến tranh và chấp nhận đàm phán tại Hội nghị Pari  
    
    Answer: [A] Phước Long was the first province to be completely liberated, and the U.S. did not intervene militarily. This proved U.S. weakness, creating confidence and reinforcing the strategic determination to fully liberate the South.  

    ---

    Question: Chiến tranh thế giới thứ hai (1939 – 1945) bùng nổ là do nguyên nhân nào sau đây?  
    A. Tác động từ sự tan rã của trật tự thế giới hai cực Ianta  
    B. Sự xuất hiện, hoạt động của Tổ chức Hiệp ước Vácsava  
    C. Sự xuất hiện của Tổ chức Hiệp ước Bắc Đại Tây Dương  
    D. Mâu thuẫn giữa các nước đế quốc về vấn đề thuộc địa  
    
    Answer: [D] The fundamental cause was the intense conflict between imperialist powers over markets and colonies. Options A, B, and C relate to the post-1945 period, not the outbreak of WWII.  

    ---

    Question: Cương lĩnh chính trị (đầu năm 1930) của Đảng Cộng sản Việt Nam... phản ánh đúng quan điểm nào sau đây của Nguyễn Ái Quốc?
    A. Lấy nhiệm vụ dân chủ làm nền tảng để giải quyết nhiệm vụ dân tộc  
    B. Giải quyết hài hòa mối quan hệ giữa vấn đề dân tộc và vấn đề giai cấp  
    C. Xác định các tầng lớp của giai cấp bóc lột đều là kẻ thù của cách mạng  
    D. Tiến hành đồng thời cách mạng phản đế và cách mạng điền địa  
    
    Answer: [B] The platform emphasized both national liberation from imperialism and land reform tasks, reflecting the viewpoint of resolving national and class issues simultaneously and harmoniously.  

    ---

    Answer the following history question: {question}  

    Options:  {choices_list}  

    Answer:""".strip()
    if is_vietnamese_model(model_name):
        instructions = fewshot_cot_vi
    else:
        instructions = fewshot_cot_en
    if is_chat_model(model_name):
        return [{"role":"user", "content":instructions}]
    return instructions
build_fewshot_cot_prompt("gemma", label.loc[0, 'question'], label.loc[0, 'choices'])

[{'role': 'user',
  'content': 'You are a history expert with deep knowledge. \n    You will be given a high school history multiple-choice question with four options.  \n    If you provide reasoning, **always give the answer first, then the explanation**.\n\n    ### OBJECTIVE\n    Analyze the question step by step. Provide ONLY ONE correct answer among the four options, and put the letter inside square brackets like: `[A]`, `[B]`, `[C]`, or `[D]`. \n\n    ### STRICT RULES\n    1. **DO NOT** repeat the question or the options. Follow this rule strictly.  \n    2. If you explain, state the answer you choose first, then the reasoning.  \n    3. Think carefully and give ONLY ONE final answer.  \n    4. Follow the sample examples below and answer in the same format.  \n\n    ### SAMPLE EXAMPLES\n    \n    ---\n\n    Question: Chiến thắng Điện Biên Phủ (1954) và thắng lợi của trận “Điện Biên Phủ trên không” (1972) ở Việt Nam đều cho thấy  \n    A. giá trị của trận quyết chiến chiến lược đán

In [7]:
import ast

def is_chat_model(model_name):
    if model_name == "google/gemma-2b" or model_name == "google/gemma-2-2b":
        return False
    return any(x in model_name.lower() for x in ["chat", "qwen", "gemma", "llama", "vistral"]) #qwen was here

def is_vietnamese_model(model_name):
    return any(x in model_name.lower() for x in ["vinallama", "vistral", "viet", "vi-"])

def build_prompt(model_name, question, choices):
    """Choose the right prompt format."""
    is_chat = is_chat_model(model_name)
    is_vi = is_vietnamese_model(model_name)
    if is_vi:
        base = prompter(question, choices)
        if is_chat:
            return [{"role":"user", "content":base}]
        else:
            return base
    else:
        base = prompter_en(question, choices)
        if is_chat:
            return [{"role":"user", "content":base}]
        else:
            return base
build_prompt("vistral", label.loc[0, 'question'], label.loc[0, 'choices'])

[{'role': 'user',
  'content': '\n### Bạn sẽ được cung cấp một câu hỏi về lịch sử. Chọn câu trả lời đúng và đặt trong ngoặc vuông như: [A], [B], [C] hoặc [D]. Hãy suy nghĩ từng bước một.\n**Không** lặp lại câu hỏi hoặc lựa chọn. Không cần giải thích lại đáp án.\nNếu bạn muốn giải thích, hãy trả lời câu hỏi bằng chữ cái [A], [B], [C] hoặc [D] trước, sau đó giải thích câu trả lời của bạn.\n\n### VÍ DỤ MẪU\n    \n---\n\nCâu hỏi: Chiến thắng Điện Biên Phủ (1954) và thắng lợi của trận “Điện Biên Phủ trên không” (1972) ở Việt Nam đều cho thấy\nA. giá trị của trận quyết chiến chiến lược đánh dấu kháng chiến kết thúc thắng lợi\nB. vai trò quyết định của mặt trận quân sự trong mối quan hệ với chính trị và ngoại giao\nC. ý nghĩa chiến lược của trận phản công lớn nhất trong cuộc chiến tranh bảo vệ Tổ quốc\nD. sức mạnh của quân chủ lực khi tấn công vào tổ chức phòng ngự mạnh của đối phương\n\nAnswer: [B] Cả hai chiến thắng này đều mang tính quyết định trong việc buộc đối phương phải chấp nhận giải

In [11]:
from tqdm import trange
import re

def extract_letter(text):
    text = text.strip()
    # 1. Prefer square bracket format [A], [B], etc.
    match = re.search(r"\[([A-D])\]", text, re.IGNORECASE)
    if match:
        return match.group(1).upper()
    # 2. Original "### Answer:" format
    match = re.search(r"###\s*Answer[:：]?\s*([A-D])\b", text, re.IGNORECASE)
    if match:
        return match.group(1).upper()
    # 3. Common patterns like "The correct answer is: B"
    match = re.search(r"(?:answer\s*(is|:)?\s*)?[\*\s]*([A-D])[\*\s]*[\.\n]", text, re.IGNORECASE)
    if match:
        return match.group(2).upper()
    # 4. First standalone A-D in first ~100 characters
    match = re.search(r"\b([A-D])\b", text[:100], re.IGNORECASE)
    if match:
        return match.group(1).upper()

    return None


def test_accuracy(model, tokenizer, *, test, model_name, batch_size=8, maxTokens=10):
    total = len(test)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = 'left'
    is_chat = is_chat_model(model_name)
    is_vilm = model_name == "vilm/vinallama-7b-chat"
    
    if is_chat and not is_vilm:
        tokenizer.chat_template = '''
        {% for message in messages %}
        {{ message['content'].strip() + '\n' }}
        {% endfor %}
        {% if add_generation_prompt %}
        {{ '' }}
        {% endif %}
        '''.strip()
        
    test_df = test.copy()
    prompts = []

    for i in range(total):
        question = test_df.loc[i, "question"]
        choices = test_df.loc[i, "choices"]
        prompt = build_prompt(model_name, question, choices) # change prompt here
        if is_vilm:
            content = prompt[0]['content'] if isinstance(prompt, list) else prompt
            text = (
                "<|im_start|>system\n"
                "Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.\n"
                "<|im_end|>\n"
                "<|im_start|>user\n"
                f"{content}<|im_end|>\n"
                "<|im_start|>assistant"
            )
            print(text)
        elif "qwen3" in model_name.lower():
            #print("Qwen3 model detected")
            text = tokenizer.apply_chat_template(
                prompt, tokenize=False, add_generation_prompt=True, enable_thinking=False
            )
        elif is_chat and isinstance(prompt, list):
            text = tokenizer.apply_chat_template(
                prompt, tokenize=False, add_generation_prompt=True
            )
        else:
            text = prompt

        prompts.append(text)

    predictions = []
    raw_responses = []

    if batch_size > total:
        print(f"WARNING: Current batch size {batch_size} is larger than number of questions. Changing to 1 instead.")
        batch_size = 1

    for i in trange(0, total, batch_size):
        batch_prompts = prompts[i:i + batch_size]
        inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True, max_length=8192).to(model.device)

        print("Max token ID:", inputs.input_ids.max().item(), "vs vocab size:", tokenizer.vocab_size)
        
        outputs = model.generate(
            **inputs,
            max_new_tokens=maxTokens,
            do_sample=False,
            top_p=None,
            top_k=None,
            temperature=None,
            pad_token_id=tokenizer.eos_token_id
        )

        generated_ids = [output[len(input_ids):] for input_ids, output in zip(inputs.input_ids, outputs)]
        decoded_outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

        print(decoded_outputs)
        raw_responses.extend(decoded_outputs)
        predictions.extend([extract_letter(text) for text in decoded_outputs])
        print(predictions)

    # Save both the full response and extracted answer
    test_df[f"{model_name}_response"] = raw_responses
    test_df[f"{model_name}_answer"] = predictions

    labels = test_df['answer'].str.strip().str.upper()
    correct = sum([p == l for p, l in zip(predictions, labels)])
    accuracy = correct / total

    return accuracy, test_df

In [9]:
import pandas as pd
from tqdm import trange
import re

def pipeline_cot_en(model_name, test):
    tokenizer, model = load_model(model_name)
    print(model.device)
    for val in [50]:
        accuracy, test_df = test_accuracy(model, tokenizer, test=test, model_name=model_name, batch_size=8, maxTokens=val)
        #print(f"Accuracy on {val} max new tokens: {accuracy}")
        safe_name = model_name.replace("/", "_")
        test_df.to_csv(f"OneshotCOT_{safe_name}_on_{val}_maxNewTokens_{accuracy}.csv", index=False)
    return accuracy, test_df
#accuracy, test_df = pipeline_cot_en("vilm/vinallama-2.7b", test)
#accuracy
#test_df

In [11]:
#9 nếu cần clear cuda khi nó hết gpu
import gc
import torch

def clear_cuda():
    gc.collect()                    # Python garbage collection
    if torch.cuda.is_available():
        torch.cuda.empty_cache()    # Releases unreferenced memory back to the pool
        torch.cuda.ipc_collect()    # Cleans up inter-process memory
clear_cuda()

In [12]:
#10
model_name = "Viet-Mistral/Vistral-7B-Chat"
accuracy, ans_df = pipeline_cot_en(model_name, label)
ans_df

Loading model: Viet-Mistral/Vistral-7B-Chat


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model successfully loaded across devices:
 - model.embed_tokens.weight on cuda:0
cuda:0


  0%|          | 0/70 [00:00<?, ?it/s]

Max token ID: 38362 vs vocab size: 38365


  0%|          | 0/70 [03:02<?, ?it/s]


KeyboardInterrupt: 

In [15]:
model_list = ["Viet-Mistral/Vistral-7B-Chat"]
#model_list = ["google/gemma-3-1b-it", "google/gemma-1.1-2b-it"]
#model_list = ["Qwen/Qwen3-4B", "Qwen/Qwen2-0.5B-Instruct", "Qwen/Qwen2.5-3B-Instruct", "Qwen/Qwen2.5-7B-Instruct", "google/gemma-3-1b-it", "google/gemma-1.1-2b-it", "google/gemma-7b-it", "vilm/vinallama-2.7b", "vilm/vinallama-7b-chat", "Viet-Mistral/Vistral-7B-Chat"]
#model_list = ["Qwen/Qwen3-4B", "Qwen/Qwen2-0.5B-Instruct", "Qwen/Qwen2.5-3B-Instruct"]
model_list = ["Qwen/Qwen2-1.5B", "Qwen/Qwen2-1.5B-Instruct", "Qwen/Qwen2-7B-Instruct", "Qwen/Qwen2.5-7B", "Qwen/Qwen3-4B", "Qwen/Qwen3-8B", "google/gemma-2b-it", "google/gemma-1.1-2b-it", "google/gemma-1.1-7b-it", "google/gemma-2-2b-it", "google/gemma-2-9b-it"]
model_list = ["google/gemma-2b-it", "google/gemma-1.1-2b-it", "google/gemma-1.1-7b-it", "google/gemma-2-2b-it", "google/gemma-2-9b-it"]
model_list = ["google/gemma-2b-it"]

for m in model_list:
    accuracy, ans_df = pipeline_cot_en(m, label)

Loading model: google/gemma-2b-it


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

['A. thành lập Mặt trận Dân chủ Đông Dương.\n\nExplanation: The 8th Central Committee of the Communist Party of Vietnam proposed the establishment of the Democratic Front, a political organization tasked with achieving national liberation.', '[C] Đánh đổ ách thống trị của đế quốc Áo – Hung.\n\nExplanation: The 15th Central Committee of the Communist Party of Vietnam (1959) adopted a policy of attacking the corrupt and oppressive regime', '[A] The correct answer is A. phân biệt chủng tộc.\n\n        Explanation: The Hiến pháp này đã quy định chế độ phân biệt chủng tộc, nghĩa là phân biệt giữa người Việt Nam và người nước ngoài.', '[C] Phe Đồng minh chưa giành được thắng lợi lớn trên chiến trường.\n\nExplanation: Ban Thường vụ Trung ương Đảng Cộng sản Đông Dương (3 – 1945) đã phát động toàn dân tổng khởi nghĩa', '[A] Huy động sức mạnh tổng hợp của toàn dân tộc để giành thắng lợi.\n\nExplanation: The West Lake campaign was a highly successful offensive that achieved a decisive victory aga

 99%|█████████▊| 69/70 [05:03<00:04,  4.42s/it]

['[A] Phong trào cách mạng 1930-1931 là phong trào diễn ra trên quy mô rộng lớn trong cả nước, trọng tâm ở các đô thị.', "[A] Hoàn thành triệt để mục tiêu của cách mạng.\n\nExplanation: The 1930-1931 Vietnamese Revolution was a significant milestone in the country's struggle for independence. It marked the beginning of a", '[A] Xuất hiện liên minh của các lực lượng cách mạng chủ lực.\n\nExplanation: The phong trào 1930-1931 là bước phát triển mới về chất so với các phong trào đấu tranh trước', '[B] Chiến dịch Điện Biên Phủ thắng lợi.\n\nExplanation: The Democratic Republic of Vietnam (DPRV) was established after the defeat of the Viet Cong in the Vietnam War. The Democratic Party of Vietnam (DPV)', '[C] Cho phép lưu hành tiền Trung Quốc.\n\nExplanation: Chính phủ Việt Nam Dân chủ Cộng hòa đã cho phép lưu hành tiền Trung Quốc cho các hộ gia đình công nhân và thương nhân trong nước, với điều kiện họ sử dụng', 'A. Nha Bình dân học vụ.\n\nExplanation: The National Student Movement in the R

100%|██████████| 70/70 [05:08<00:00,  4.41s/it]

['[C] Vấn đề ruộng đất của nông dân được giải quyết triệt để.\n\nExplanation: The question asked about the development of the national defense system in Vietnam during the 1950s and 1953 compared', '[C] Góp phần phát triển hậu phương, tăng cường tiềm lực cách mạng.\n\nExplanation: The phong trào đấu tranh chống Mĩ của nhân dân miền Nam Việt Nam giai đoạn 1961 – 196', '[C] Chiến tranh cục bộ.\n\nExplanation: The question asked about the US military strategy in Vietnam from 1965 to 1968. The passage does not provide any information about the US military strategy, so the', '[A] Thực dân Pháp bị thiệt hại nặng nề. \n\nExplanation: The Nava Plan was conceived in response to the heavy losses suffered by the French military in the Battle of the Pacific and the subsequent retreat from Southeast Asia. The', '[C] Quân Pháp bị sa lầy trên chiến trường.\n\nExplanation: The question asks about the reason behind the decision to implement the Nava Plan. The correct answer is C, indicating that the Fr

In [17]:
ans_df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,answer,question_type,grade,choices,question,nhan_biet,thong_hieu,van_dung,van_dung_cao,dang_cau_hoi,Qwen/Qwen3-8b_response,Qwen/Qwen3-8b_answer
0,history-2022-302-35,A,multiple_choice,12.0,"['Có sự linh hoạt, sáng tạo trong phương án tá...",Chiến dịch Biên giới thu-đông (1950) và chiến ...,NaN,NaN,1.0,NaN,"So sánh, đối chiếu",[A] \nChiến dịch Biên giới thu,A
1,history-2022-304-37,A,multiple_choice,12.0,['Sử dụng đòn tiến công chiến lược của lực lượ...,Chiến dịch Biên giới thu-đông (1950) và chiến ...,NaN,NaN,1.0,NaN,"So sánh, đối chiếu","[Answer]\n[A] \n\nThe two campaigns,",A
2,history-2020-304-21,A,multiple_choice,12.0,"['sự bất lực của quân đội Sài Gòn', 'nhiệm vụ ...",Chiến thắng Phước Long (tháng 1-1975) của quân...,NaN,NaN,1.0,NaN,Đánh giá,[Answer]\n[D] \n\nThe victory at Ph,D
3,history-2020-301-32,A,multiple_choice,12.0,"['khả năng thắng lớn của quân giải phóng.', 'q...",Chiến thắng Phước Long (tháng 1-1975) của quân...,NaN,NaN,1.0,NaN,Đánh giá,[ A ]\n\n[ A ] because the victory at,A
4,history-2020-302-23,A,multiple_choice,12.0,"['sức mạnh to lớn của quân giải phóng.', 'quân...",Chiến thắng Phước Long (tháng 1-1975) của quân...,NaN,NaN,1.0,NaN,Đánh giá,[C] \nTrận chiến Phước Long năm,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,history-2021-304-9,A,multiple_choice,12.0,"['Thực dân Pháp bị thiệt hại nặng nề', 'Nhật đ...","Ở Đông Dương, thực dân Pháp đề ra kế hoạch quâ...",NaN,1.0,NaN,NaN,Bối cảnh/Nguyên nhân,[Answer]: [D]\n\n### Explanation:\nK,D
556,history-2021-301-11,C,multiple_choice,12.0,"['Quân phiệt Nhật Bản tiến vào Đông Dương.', '...","Ở Đông Dương, thực dân Pháp đề ra kế hoạch quâ...",NaN,1.0,NaN,NaN,Bối cảnh/Nguyên nhân,[C] \nC. Quân Pháp bị sa l,C
557,history-2021-302-3,B,multiple_choice,12.0,"['Quân phiệt Nhật Bản tiến vào Đông Dương.', '...","Ở Đông Dương, thực dân Pháp đề ra kế hoạch quâ...",NaN,1.0,NaN,NaN,Bối cảnh/Nguyên nhân,[Answer]: [B] \n**Explanation**:,B
558,history-2021-303-2,C,multiple_choice,12.0,"['Quân Trung Hoa Dân quốc tiến vào Việt Nam.',...","Ở Đông Dương, thực dân Pháp đề ra kế hoạch quâ...",NaN,1.0,NaN,NaN,Bối cảnh/Nguyên nhân,[ C ]\n\nThe French colonial authorities in In...,C


In [18]:
accuracy

0.7357142857142858

In [15]:
#clear kaggle working directory to save disk space
print("Are you sure")
a = int(input())
if a == 123:
    !rm -rf /kaggle/working/*
else:
    print("Ok good just making sure")

Are you sure bro


 123


In [30]:
#gemma3-4b-it required a different inference method
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
import torch

def load_gemma3_model(model_name):
    print(f"Loading gemma3 model: {model_name}")
    clear_cuda()
    model = Gemma3ForConditionalGeneration.from_pretrained(
        model_name, 
        torch_dtype="float16",
        device_map="auto"
    ).eval()
    processor = AutoProcessor.from_pretrained(model_name)
    print("Model successfully loaded across devices:")
    for name, param in model.named_parameters():
       if param.device.type == 'cuda': 
           print(f" - {name} on {param.device}")
           break  # print only one to verify
    return processor, model

In [45]:
from tqdm import trange
import re
import torch

def test_gemma3_accuracy(model, processor, *, test, model_name, batch_size=4, maxTokens=50):
    """
    Evaluate Gemma-3 IT on multiple-choice dataset with batch inference.
    Saves both raw responses and extracted answers into the dataframe.
    """
    total = len(test)
    test_df = test.copy()

    prompts = []
    for i in range(total):
        question = test_df.loc[i, "question"]
        choices = test_df.loc[i, "choices"]
        # Reuse your own prompter function if needed
        user_text = prompter_en(question, choices)

        messages = [
            {"role": "system", "content": [{"type": "text", "text": "You are an expert on Vietnamese history."}]},
            {"role": "user",   "content": [{"type": "text", "text": user_text}]}
        ]
        prompts.append(messages)
    print(messages)
    predictions = []
    raw_responses = []

    if batch_size > total:
        print(f"WARNING: Current batch size {batch_size} > total {total}. Using 1 instead.")
        batch_size = 1

    for i in trange(0, total, batch_size):
        batch_prompts = prompts[i:i + batch_size]

        # Apply chat template to *all* prompts in batch
        inputs = processor.apply_chat_template(
            batch_prompts,
            add_generation_prompt=True,
            tokenize=True,
            return_tensors="pt",
            return_dict=True,
            padding=True,
            truncation=True,
        ).to(model.device, dtype=torch.float16)

        input_len = inputs["input_ids"].shape[-1]

        with torch.inference_mode():
            outputs = model.generate(
                **inputs,
                max_new_tokens=maxTokens,
                do_sample=False,
                top_p=0,
                top_k=0,
                temperature=None,
                pad_token_id=processor.tokenizer.eos_token_id
            )

        # Slice away input part and decode each sample
        batch_decoded = []
        for input_ids, output_ids in zip(inputs.input_ids, outputs):
            gen_ids = output_ids[len(input_ids):]
            text = processor.decode(gen_ids, skip_special_tokens=True).strip()
            batch_decoded.append(text)

        raw_responses.extend(batch_decoded)

        # Extract answer letters
        for text in batch_decoded:
            match = re.search(r"\b([A-D])\b", text, re.IGNORECASE)
            predictions.append(match.group(1).upper() if match else None)

    # Save into dataframe
    test_df[f"{model_name}_response"] = raw_responses
    test_df[f"{model_name}_answer"] = predictions

    labels = test_df['answer'].str.strip().str.upper()
    correct = sum([p == l for p, l in zip(predictions, labels)])
    accuracy = correct / total

    return accuracy, test_df


In [46]:
#processor, model = load_gemma3_model("google/gemma-3-4b-it")
accuracy, ans_df = test_gemma3_accuracy(model, processor, test=label.head(8), model_name="google/gemma-3-4b-it", batch_size=4, maxTokens=10)

[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are an expert on Vietnamese history.'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': '### Answer this history question:\nChiến thắng Việt Bắc thu-đông năm 1947 của quân dân Việt Nam đã\n### Here are your choices:\nA. làm thất bại chiến tranh tổng lực.\nB. làm thất bại chiến tranh cục bộ.\nC. làm thất bại chiến tranh đặc biệt.\nD. cho thấy bộ đội chủ lực ngày càng trưởng thành.\n### Choose the correct answer and put it inside square brackets like: [A], [B], [C], or [D]. \nDo **NOT** repeat the question or choices. You do NOT need explain your answer. \nIf you are going to explain yourself, answer the question with a letter [A], [B], [C] or [D] first, then explain your answer.\n### Answer: '}]}]


100%|██████████| 2/2 [00:01<00:00,  1.14it/s]


In [35]:
accuracy

0.0

In [47]:
ans_df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,answer,question_type,grade,choices,question,nhan_biet,thong_hieu,van_dung,van_dung_cao,dang_cau_hoi,google/gemma-3-4b-it_response,google/gemma-3-4b-it_answer
0,history-2022-302-35,A,multiple_choice,12.0,"['Có sự linh hoạt, sáng tạo trong phương án tá...",Chiến dịch Biên giới thu-đông (1950) và chiến ...,NaN,NaN,1.0,NaN,"So sánh, đối chiếu",,None
1,history-2022-304-37,A,multiple_choice,12.0,['Sử dụng đòn tiến công chiến lược của lực lượ...,Chiến dịch Biên giới thu-đông (1950) và chiến ...,NaN,NaN,1.0,NaN,"So sánh, đối chiếu",,None
2,history-2020-304-21,A,multiple_choice,12.0,"['sự bất lực của quân đội Sài Gòn', 'nhiệm vụ ...",Chiến thắng Phước Long (tháng 1-1975) của quân...,NaN,NaN,1.0,NaN,Đánh giá,,None
3,history-2020-301-32,A,multiple_choice,12.0,"['khả năng thắng lớn của quân giải phóng.', 'q...",Chiến thắng Phước Long (tháng 1-1975) của quân...,NaN,NaN,1.0,NaN,Đánh giá,,None
4,history-2020-302-23,A,multiple_choice,12.0,"['sức mạnh to lớn của quân giải phóng.', 'quân...",Chiến thắng Phước Long (tháng 1-1975) của quân...,NaN,NaN,1.0,NaN,Đánh giá,,None
5,history-2020-303-25,B,multiple_choice,12.0,"['quân đội Sài Gòn đã tan rã hoàn toàn.', 'sự ...",Chiến thắng Phước Long (tháng 1-1975) của quân...,NaN,NaN,1.0,NaN,Đánh giá,,None
6,history-2020-303-11,D,multiple_choice,12.0,"['làm thất bại chiến tranh cục bộ.', 'làm thất...",Chiến thắng Việt Bắc thu-đông năm 1947 của quâ...,NaN,1.0,NaN,NaN,Kết quả / Ý nghĩa / Vai trò,,None
7,history-2020-301-1,D,multiple_choice,12.0,"['làm thất bại chiến tranh tổng lực.', 'làm th...",Chiến thắng Việt Bắc thu-đông năm 1947 của quâ...,NaN,1.0,NaN,NaN,Kết quả / Ý nghĩa / Vai trò,,None


In [ ]:
# pip install accelerate

from transformers import AutoProcessor, Gemma3ForConditionalGeneration
from PIL import Image
import requests
import torch

model_id = "google/gemma-3-4b-it"

model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id, device_map="auto", offload_buffers=True
).eval()
processor = AutoProcessor.from_pretrained(model_id)

# **Overall Impression:** The image is a close-up shot of a vibrant garden scene,
# focusing on a cluster of pink cosmos flowers and a busy bumblebee.
# It has a slightly soft, natural feel, likely captured in daylight.


In [ ]:
import ast
import re
from tqdm import trange
import torch

def extract_letter(text):
  text = text.strip()
  # 1. Prefer square bracket format [A], [B], etc.
  match = re.search(r"\[([A-D])\]", text, re.IGNORECASE)
  if match:
      return match.group(1).upper()
  # 2. Original "### Answer:" format
  match = re.search(r"###\s*Answer[:：]?\s*([A-D])\b", text, re.IGNORECASE)
  if match:
      return match.group(1).upper()
  # 3. Common patterns like "The correct answer is: B"
  match = re.search(r"(?:answer\s*(is|:)?\s*)?[\*\s]*([A-D])[\*\s]*[\.\n]", text, re.IGNORECASE)
  if match:
      return match.group(2).upper()
  # 4. First standalone A-D in first ~100 characters
  match = re.search(r"\b([A-D])\b", text[:100], re.IGNORECASE)
  if match:
      return match.group(1).upper()

  return None

def test_gemma3_accuracy_batched(label_df, maxNewTokens, batch_size):
  total = len(label_df)
  ans_df = label_df.copy()
  ans_df["Model Inference"] = None
  ans_df["Model Answer"] = None

  for i in trange(0, total, batch_size):
    batch_indices = range(i, min(i + batch_size, total))
    batch_messages_text = []  # List to hold the formatted message strings
    for j in batch_indices:
        question = label_df.loc[j, 'question']
        choices = label_df.loc[j, 'choices']
        if isinstance(choices, str):
              choices_list = "\n".join([f"{chr(65+k)}. {c}" for k,c in enumerate(ast.literal_eval(choices))])
        else:
              choices_list = "\n".join([f"{chr(65+k)}. {c}" for k,c in enumerate(choices)])
        messages = [
            {
                "role": "system",
                "content": [{"type": "text", "text": "You are a helpful assistant."}]
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": f"""
    You are a history expert with deep knowledge.
    You will be given a high school history multiple-choice question with four options.
    If you provide reasoning, **always give the answer first, then the explanation**.

    ### OBJECTIVE
    Analyze the question step by step. Provide ONLY ONE correct answer among the four options, and put the letter inside square brackets like: `[A]`, `[B]`, `[C]`, or `[D]`.

    ### STRICT RULES
    1. **DO NOT** repeat the question or the options. Follow this rule strictly.
    2. If you explain, state the answer you choose first, then the reasoning.
    3. Think carefully and give ONLY ONE final answer.
    4. Follow the sample examples below and answer in the same format.

    ### SAMPLE EXAMPLES

    ---

    Question: Chiến thắng Điện Biên Phủ (1954) và thắng lợi của trận “Điện Biên Phủ trên không” (1972) ở Việt Nam đều cho thấy
    A. giá trị của trận quyết chiến chiến lược đánh dấu kháng chiến kết thúc thắng lợi
    B. vai trò quyết định của mặt trận quân sự trong mối quan hệ với chính trị và ngoại giao
    C. ý nghĩa chiến lược của trận phản công lớn nhất trong cuộc chiến tranh bảo vệ Tổ quốc
    D. sức mạnh của quân chủ lực khi tấn công vào tổ chức phòng ngự mạnh của đối phương

    Answer: [B] Both victories were decisive in forcing the opponent to accept unfavorable political and diplomatic solutions. This shows that the military front played the decisive role, laying the foundation for political and diplomatic victories.

    ---

    Question: Chiến thắng Điện Biên Phủ năm 1954 của quân dân Việt Nam đã làm phá sản hoàn toàn kế hoạch nào sau đây của thực dân Pháp?
    A. Rove
    B. Bôlae
    C. Đờ Lát đơ Tátxinhi
    D. Nava

    Answer: [D] Điện Biên Phủ destroyed the Navarre Plan—the last major French military strategy aimed at achieving decisive victory within 18 months. The Rove, Bôlae, and De Lattre plans had already been defeated earlier.

    ---

    Question: Chiến thắng Đường 14-Phước Long (đầu năm 1975) có tác động nào sau đây đối với tiến trình cách mạng miền Nam Việt Nam?
    A. Củng cố quyết tâm chiến lược giải phóng hoàn toàn miền Nam
    B. Chuyển cách mạng miền Nam từ thế giữ gìn lực lượng sang thế tiến công
    C. Lần đầu khẳng định bạo lực là con đường giải phóng miền Nam
    D. Buộc Mĩ xuống thang chiến tranh và chấp nhận đàm phán tại Hội nghị Pari

    Answer: [A] Phước Long was the first province to be completely liberated, and the U.S. did not intervene militarily. This proved U.S. weakness, creating confidence and reinforcing the strategic determination to fully liberate the South.

    ---

    Question: Chiến tranh thế giới thứ hai (1939 – 1945) bùng nổ là do nguyên nhân nào sau đây?
    A. Tác động từ sự tan rã của trật tự thế giới hai cực Ianta
    B. Sự xuất hiện, hoạt động của Tổ chức Hiệp ước Vácsava
    C. Sự xuất hiện của Tổ chức Hiệp ước Bắc Đại Tây Dương
    D. Mâu thuẫn giữa các nước đế quốc về vấn đề thuộc địa

    Answer: [D] The fundamental cause was the intense conflict between imperialist powers over markets and colonies. Options A, B, and C relate to the post-1945 period, not the outbreak of WWII.

    ---

    Question: Cương lĩnh chính trị (đầu năm 1930) của Đảng Cộng sản Việt Nam... phản ánh đúng quan điểm nào sau đây của Nguyễn Ái Quốc?
    A. Lấy nhiệm vụ dân chủ làm nền tảng để giải quyết nhiệm vụ dân tộc
    B. Giải quyết hài hòa mối quan hệ giữa vấn đề dân tộc và vấn đề giai cấp
    C. Xác định các tầng lớp của giai cấp bóc lột đều là kẻ thù của cách mạng
    D. Tiến hành đồng thời cách mạng phản đế và cách mạng điền địa

    Answer: [B] The platform emphasized both national liberation from imperialism and land reform tasks, reflecting the viewpoint of resolving national and class issues simultaneously and harmoniously.

    ---

    Answer the following history question: {question}

    Options:  {choices_list}

    Answer:"""}
                ]
            }
        ]
        # Apply chat template and get the formatted string for each message
        formatted_message = processor.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
        batch_messages_text.append(formatted_message)


    # Process the list of formatted message strings
    # Build batch inputs
    inputs = processor(
        text=batch_messages_text, padding=True, return_tensors="pt"
    ).to(model.device, dtype=torch.bfloat16)

    # Track original lengths (before padding)
    input_lengths = [len(processor.tokenizer.encode(msg)) for msg in batch_messages_text]

    with torch.inference_mode():
        generation = model.generate(**inputs, max_new_tokens=maxNewTokens, do_sample=False, top_p=None, top_k=None)

    for k, idx in enumerate(batch_indices):
        generated_text = generation[k][input_lengths[k]:].tolist()
        decoded = processor.tokenizer.decode(generated_text, skip_special_tokens=True)
        ans_df.loc[idx, "Model Inference"] = decoded
        ans_df.loc[idx, "Model Answer"] = extract_letter(decoded)


  accuracy = (ans_df['Model Answer'] == ans_df['answer'].str.strip().str.upper()).mean()
  return ans_df, accuracy


ans_df, accuracy2 = test_gemma3_accuracy_batched(label, 50, batch_size=8)
ans_df
ans_df.to_csv(f'Fewshot_Gemma3_4b_it_on_50maxnewtokens_{accuracy2}.csv')

In [ ]:
accuracy = accuracy = (ans_df['Model Answer'] == ans_df['answer'].str.strip().str.upper()).mean()